In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("insurance.csv")
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [3]:
df.shape

(1338, 7)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [5]:
# Check for null values
df.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [6]:
cat_var = [x for x in df.columns if df[x].dtypes == 'object']
cat_var

['sex', 'smoker', 'region']

In [7]:
# Using Label Encoder
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
for col in cat_var:
    df[col] = enc.fit_transform(df[col])

In [8]:
df

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,1,21984.47061
4,32,1,28.880,0,0,1,3866.85520
...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,1,10600.54830
1334,18,0,31.920,0,0,0,2205.98080
1335,18,0,36.850,0,0,2,1629.83350
1336,21,0,25.800,0,0,3,2007.94500


In [9]:
# Scaling : min max scaling
data = df.drop('charges', axis = 1)
data = data.apply(lambda x: (x - x.min())/(x.max() - x.min()))
data['charges'] = df.charges

In [10]:
x = data.drop('charges', axis = 1).to_numpy()
y = df.charges.to_numpy()

In [84]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 0)

In [85]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1003, 6)
(1003,)
(335, 6)
(335,)


In [86]:
X_train = torch.tensor(X_train, dtype = torch.float32)
X_test = torch.tensor(X_test, dtype = torch.float32)
y_train = torch.tensor(y_train, dtype = torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype = torch.float32).view(-1, 1)

In [87]:
X_train

tensor([[0.3043, 0.0000, 0.3667, 0.2000, 0.0000, 0.6667],
        [0.9348, 0.0000, 0.1636, 0.0000, 0.0000, 0.0000],
        [0.0217, 1.0000, 0.2324, 0.2000, 0.0000, 1.0000],
        ...,
        [0.4783, 1.0000, 0.2454, 0.0000, 0.0000, 0.6667],
        [0.0217, 1.0000, 0.5265, 0.0000, 0.0000, 0.3333],
        [0.3261, 0.0000, 0.0683, 0.2000, 0.0000, 1.0000]])

In [88]:
X_test

tensor([[0.7391, 1.0000, 0.3831, 0.2000, 0.0000, 1.0000],
        [0.6304, 0.0000, 0.3608, 0.2000, 0.0000, 0.6667],
        [0.6522, 1.0000, 0.6620, 0.4000, 1.0000, 0.3333],
        ...,
        [0.7174, 1.0000, 0.3785, 0.2000, 0.0000, 0.6667],
        [0.1087, 1.0000, 0.5687, 0.6000, 0.0000, 1.0000],
        [0.9348, 0.0000, 0.3527, 0.0000, 1.0000, 0.3333]])

In [89]:
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [90]:
data = TensorDataset(X_train, y_train)

In [91]:
batch_size = 10
train_dl = DataLoader(data, batch_size, shuffle = True)

In [92]:
# Creating a model : it will just generate random parameters
x_num = X_train.shape[1]

model = nn.Linear(x_num, 1)

In [93]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3358,  0.3467,  0.2750, -0.0371,  0.3468, -0.0123]],
        requires_grad=True),
 Parameter containing:
 tensor([0.0973], requires_grad=True)]

In [94]:
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.3358,  0.3467,  0.2750, -0.0371,  0.3468, -0.0123]],
       requires_grad=True)
Parameter containing:
tensor([0.0973], requires_grad=True)


In [95]:
# Create a loss function
import torch.nn.functional as f
loss_fn = f.mse_loss

In [96]:
loss_fn(model(X_train), y_train)

tensor(3.1753e+08, grad_fn=<MseLossBackward0>)

In [97]:
X_train.dtype

torch.float32

In [98]:
model.weight.dtype

torch.float32

In [100]:
# The prediction will still be very poor
preds = model(X_train)
preds

tensor([[0.2848],
        [0.4562],
        [0.4956],
        ...,
        [0.6639],
        [0.5920],
        [0.2059]], grad_fn=<AddmmBackward0>)

In [101]:
# Creating an optimizer 
optimizer = torch.optim.SGD(model.parameters(), lr= 1e-5)

# Train the model

In [106]:
def fit(model, epochs, train_dl, optimizer, loss):
    # Looping through the number of epoch
    for epoch in range(epochs):
        for xb, yb in train_dl:
            # Generate predictions
            pred = model(xb)

            # Calculate the loss
            loss = loss_fn(pred, yb)
            
            # compute the gradient values wrt to weights and biases
            loss.backward()
            
            # optimize
            optimizer.step()
            
            # Clear the gradients
            optimizer.zero_grad()
            
        if (epoch + 1) % 10 == 0:
            print(f"[{epoch + 1}/{epochs}], Loss : {loss}")

In [108]:
history = fit(model, 1000, train_dl, optimizer, loss)

[10/1000], Loss : 2220789.75
[20/1000], Loss : 402751872.0
[30/1000], Loss : 124801824.0
[40/1000], Loss : 69706072.0
[50/1000], Loss : 6497395.5
[60/1000], Loss : 224958192.0
[70/1000], Loss : 27674782.0
[80/1000], Loss : 255967552.0
[90/1000], Loss : 658821632.0
[100/1000], Loss : 295235840.0
[110/1000], Loss : 8703383.0
[120/1000], Loss : 18144248.0
[130/1000], Loss : 151346976.0
[140/1000], Loss : 16759432.0
[150/1000], Loss : 8902080.0
[160/1000], Loss : 17663462.0
[170/1000], Loss : 283109472.0
[180/1000], Loss : 100088032.0
[190/1000], Loss : 27886872.0
[200/1000], Loss : 59545648.0
[210/1000], Loss : 49159916.0
[220/1000], Loss : 855858176.0
[230/1000], Loss : 217513472.0
[240/1000], Loss : 142852544.0
[250/1000], Loss : 267628672.0
[260/1000], Loss : 132266144.0
[270/1000], Loss : 443822336.0
[280/1000], Loss : 197813520.0
[290/1000], Loss : 19762182.0
[300/1000], Loss : 23134674.0
[310/1000], Loss : 120475944.0
[320/1000], Loss : 340202208.0
[330/1000], Loss : 128370600.0
[34